In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import ast
writer = pd.ExcelWriter('football_players.xlsx', engine='xlsxwriter')
writer.save()
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
sparql.setTimeout(5000)

In [2]:
def get_json_football_player_data(league):
    #Final Working version to get teams
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?name ?clubname (concat('[',group_concat(distinct ?birthPlace;separator=','),']') as ?birthPlace) (concat('[',group_concat(distinct ?longitude;separator=','),']') as ?longitude) (concat('[',group_concat(distinct ?latitude;separator=','),']') as ?latitude) (concat('["',group_concat(distinct ?num_clubs;separator='","'),'"]') as ?num_clubs) (concat('[',group_concat(distinct ?positionlabel;separator=','),']') as ?positionlabel) (year(xsd:dateTime(?birthDate)) as ?birthDate) ?height (concat('[',group_concat(distinct ?goals;separator=','),']') as ?goals)  (concat('[',group_concat(distinct ?nationalgoals;separator=','),']') as ?nationalgoals) (concat('[',group_concat(distinct ?nationalcaps;separator=','),']') as ?nationalcaps) (concat('[',group_concat(distinct ?clubcaps;separator=','),']') as ?clubcaps) 
    WHERE {    
    ?team <http://purl.org/dc/terms/subject> """ + league + """ .
    ?team dbp:clubname ?clubname .

    ?players dbp:name ?name .
    ?players dbo:birthPlace ?birthPlace .
    ?birthPlace geo:lat ?latitude .
    ?birthPlace geo:long ?longitude .
    ?players dbp:currentclub ?team .
    ?players dbp:birthDate ?birthDate .
    ?players dbp:height ?height .
    ?players dbp:caps ?clubcaps .
    ?players dbp:position ?position .
    ?position rdfs:label ?positionlabel .

    OPTIONAL {?players dbp:goals ?goals} .
    OPTIONAL {?players dbp:nationalgoals ?nationalgoals} .
    OPTIONAL {?players dbp:nationalcaps ?nationalcaps  } .
    OPTIONAL {?players dbp:clubs ?num_clubs }.
    FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
    FILTER(datatype(?goals) = <http://www.w3.org/2001/XMLSchema#integer>) .
    FILTER(datatype(?nationalgoals) = <http://www.w3.org/2001/XMLSchema#integer>) .
    FILTER(datatype(?clubcaps) = <http://www.w3.org/2001/XMLSchema#integer>) .
    FILTER(datatype(?nationalcaps) = <http://www.w3.org/2001/XMLSchema#integer>) .
    FILTER (LANGMATCHES(LANG(?positionlabel), 'en'))
    }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']



In [3]:
prem_data = get_json_football_player_data("<http://dbpedia.org/resource/Category:Premier_League_clubs>")

In [4]:
laliga_data = get_json_football_player_data("<http://dbpedia.org/resource/Category:La_Liga_clubs>")

In [5]:
def get_all_players(prem_data,laliga_data):
    player_attributes_dct =  {"name":[],"latitude":[],"longitude":[],"num_clubs":[],"clubname":[], "league":[], "birthPlace":[], "positionlabel":[], "birthDate":[], "height":[], "nationalgoals":[], "goals":[], "nationalcaps":[], "clubcaps":[]}
    leagues = [prem_data,laliga_data]
    type_string = ["Premier League","La Liga"]
    index = 0
    for league in leagues:
        for team in league:
            player_attributes_dct["name"].append(team['name']['value'])
            player_attributes_dct["longitude"].append(team['longitude']['value'])
            player_attributes_dct["latitude"].append(team['latitude']['value'])
            player_attributes_dct["clubname"].append(team['clubname']['value'])
            player_attributes_dct["num_clubs"].append(len(ast.literal_eval(team['num_clubs']['value'])))
            player_attributes_dct["positionlabel"].append(team['positionlabel']['value'])                  
            player_attributes_dct["league"].append(type_string[index])
            player_attributes_dct["birthPlace"].append(team['birthPlace']['value'])
            player_attributes_dct["birthDate"].append(team['birthDate']['value'])
            player_attributes_dct["height"].append(team['height']['value'])
            player_attributes_dct["nationalgoals"].append(sum(ast.literal_eval(team['nationalgoals']['value'])))
            player_attributes_dct["goals"].append(sum(ast.literal_eval(team['goals']['value'])))
            player_attributes_dct["nationalcaps"].append(sum(ast.literal_eval(team['nationalcaps']['value'])))
            player_attributes_dct["clubcaps"].append(sum(ast.literal_eval(team['clubcaps']['value'])))
            
        index += 1
    return player_attributes_dct


In [6]:
dct1 = get_all_players(prem_data,laliga_data)


In [7]:
import pandas as pd


df1 = pd.DataFrame(dct1)

writer = pd.ExcelWriter('football_players.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df1.to_excel(writer, sheet_name='Sheet1_Football_players', index=False)


# Close the Pandas Excel writer and output the Excel file.
writer.save()